In [ ]:
#!python3 -m pip install --force-reinstall sql_sidekick-0.1.8-py3-none-any.whl

In [57]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [58]:
from sidekick.prompter import db_setup, ask
from sidekick.schema_generator import generate_schema
from sidekick.utils import setup_dir

In [59]:
base_path = "/home/pramit/testing/q/apps/system/sql-sidekick"
cache_path = f"{base_path}/var/lib/tmp"
setup_dir(base_path)

In [60]:
# Setup the DB for a single table (WIP to extend to multiple tables)

HOST_NAME = "localhost"
USER_NAME = "sqlite"
PASSWORD = "abc"
DB_NAME = "querydb"
PORT = "5432"


# Given .csv file, auto-generate schema
# Download dataset --> https://www.kaggle.com/datasets/uom190346a/sleep-health-and-lifestyle-dataset
data_path = "./sleep_health_and_lifestyle_dataset.csv"
table_name = "sleep_health_eda_demo"

r, table_info_path = generate_schema(data_path, f"{cache_path}/{table_name}_table_info.jsonl")

llm_response, err = db_setup(
                db_name=DB_NAME,
                hostname=HOST_NAME,
                user_name=USER_NAME,
                password=PASSWORD,
                port=PORT,
                table_info_path=table_info_path,
                table_samples_path=data_path,
                table_name=table_name,
                local_base_path=base_path
            )

 Information supplied:
 querydb, localhost, sqlite, abc, 5432


2023-12-01 01:19:13.600 | DEBUG    | sidekick.db_config:__init__:42 - Creating SQLite DB: sqlite:////home/pramit/testing/q/apps/system/sql-sidekick/db/sqlite/querydb.db


Database already exists!
Table name: sleep_health_eda_demo


2023-12-01 01:19:13.607 | DEBUG    | sidekick.db_config:_extract_schema_info:100 - Schema path: /home/pramit/testing/q/apps/system/sql-sidekick/var/lib/tmp/sleep_health_eda_demo_table_info.jsonl
2023-12-01 01:19:13.611 | DEBUG    | sidekick.db_config:create_table:145 - Schema info used for creating table:
 Person_ID NUMERIC,
Gender TEXT COLLATE NOCASE,
Age NUMERIC,
Occupation TEXT COLLATE NOCASE,
Sleep_Duration NUMERIC,
Quality_of_Sleep NUMERIC,
Physical_Activity_Level NUMERIC,
Stress_Level NUMERIC,
BMI_Category TEXT COLLATE NOCASE,
Blood_Pressure TEXT COLLATE NOCASE,
Heart_Rate NUMERIC,
Daily_Steps NUMERIC,
Sleep_Disorder TEXT COLLATE NOCASE


Checked table sleep_health_eda_demo exists in the DB.


2023-12-01 01:19:13.626 | DEBUG    | sidekick.db_config:add_samples:179 - Adding sample values to table: /home/pramit/testing/q/apps/system/sql-sidekick/examples/demo/sleep_health_and_lifestyle_dataset.csv
2023-12-01 01:19:13.632 | DEBUG    | sidekick.db_config:add_samples:185 - Inserting chunk: 0
2023-12-01 01:19:13.723 | INFO     | sidekick.db_config:add_samples:190 - Data inserted into table: sleep_health_eda_demo
2023-12-01 01:19:13.727 | INFO     | sidekick.db_config:add_samples:195 - Number of rows inserted: 374


Created a Database querydb. Inserted sample values from /home/pramit/testing/q/apps/system/sql-sidekick/examples/demo/sleep_health_and_lifestyle_dataset.csv into table sleep_health_eda_demo, please ask questions!


In [61]:
def query(
    question: str,
    table_name,
    table_info_path: str,
    sample_qna_path: str,
    regenerate=False,
    regenerate_with_options=False,
):
    """Asks question and returns SQL."""
    base_path = "./"
    # self_correction is enabled by default, set to False if not needed.
    res = ask(
        question=question,
        table_info_path=table_info_path,
        sample_queries_path=sample_qna_path,
        table_name=table_name,
        is_command=False,
        model_name="h2ogpt-sql-nsql-llama-2-7B",  #Other default model option: h2ogpt-sql-sqlcoder2
        is_regenerate=regenerate,
        is_regen_with_options=regenerate_with_options,
        execute_query=False,
        local_base_path=base_path,
    )
    return res

In [62]:
res = query("What is the average sleep duration for each gender?", table_name="sleep_health_eda_demo", 
            table_info_path=table_info_path, sample_qna_path=None)

2023-12-01 01:19:14.884 | INFO     | sidekick.prompter:ask:428 - Table in use: ['sleep_health_eda_demo']
2023-12-01 01:19:14.886 | INFO     | sidekick.prompter:ask:460 - Setting context...
2023-12-01 01:19:14.888 | INFO     | sidekick.prompter:ask:461 - Question: What is the average sleep duration for each gender?
2023-12-01 01:19:14.889 | INFO     | sidekick.utils:is_resource_low:324 - Number of GPUs: 1
2023-12-01 01:19:14.891 | DEBUG    | sidekick.utils:is_resource_low:328 - Information on device: 0
2023-12-01 01:19:14.892 | INFO     | sidekick.utils:is_resource_low:331 - Total Memory: 23GB
2023-12-01 01:19:14.894 | INFO     | sidekick.utils:is_resource_low:332 - Free GPU memory: 9GB
2023-12-01 01:19:14.955 | INFO     | sidekick.utils:_check_file_info:438 - Using information info from path /home/pramit/testing/q/apps/system/sql-sidekick/var/lib/tmp/sleep_health_eda_demo_table_info.jsonl
2023-12-01 01:19:14.979 | DEBUG    | sidekick.utils:semantic_search:125 - Input questions: # query

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-12-01 01:19:15.044 | DEBUG    | sidekick.utils:semantic_search:130 - Probable context: ["if patterns like 'current time' or 'now' occurs in question", "if patterns like 'total number', or 'list' occurs in question", 'detailed summary', 'summary']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-12-01 01:19:15.084 | DEBUG    | sidekick.utils:semantic_search:138 - Similarity score for: if patterns like 'current time' or 'now' occurs in question: 0.8459207869447033
2023-12-01 01:19:15.087 | DEBUG    | sidekick.utils:semantic_search:138 - Similarity score for: if patterns like 'total number', or 'list' occurs in question: 0.8319947353454415
2023-12-01 01:19:15.089 | DEBUG    | sidekick.utils:semantic_search:138 - Similarity score for: detailed summary: 0.8346069603076574
2023-12-01 01:19:15.091 | DEBUG    | sidekick.utils:semantic_search:138 - Similarity score for: summary: 0.8394152180082535
2023-12-01 01:19:15.093 | DEBUG    | sidekick.utils:semantic_search:154 - Sorted context: []
2023-12-01 01:19:15.094 | DEBUG    | sidekick.query:generate_sql:402 - Filter Context: []
2023-12-01 01:19:15.095 | INFO     | sidekick.query:generate_sql:410 - Filtering Question/Query pairs ...
2023-12-01 01:19:15.096 | INFO     | sidekick.query:generate_sql:412 - Number of context queries fou

Exiting...


In [63]:
print(f"Question = {res[0][0]}")
print("----")
print(f"Generated SQL = {res[0][1]}")

question = **Generated response for question,**
What is the average sleep duration for each gender?

----
Generated SQL = ``` sql
SELECT AVG("Sleep_Duration"),
       "Gender"
FROM "sleep_health_eda_demo"
GROUP BY "Gender"
LIMIT 100
```




In [64]:
# On using re-generation flag we toggle the temperature values between 0 and 1 alternating between low 
# (focus/conservative generation and high values (random/creative generation)
res = query("What is the average sleep duration for each gender?", table_name="sleep_health_eda_demo", 
            table_info_path=table_info_path, sample_qna_path=None, regenerate=True)

2023-12-01 01:19:19.655 | INFO     | sidekick.prompter:ask:428 - Table in use: ['sleep_health_eda_demo']
2023-12-01 01:19:19.657 | INFO     | sidekick.prompter:ask:460 - Setting context...
2023-12-01 01:19:19.658 | INFO     | sidekick.prompter:ask:461 - Question: What is the average sleep duration for each gender?
2023-12-01 01:19:19.660 | INFO     | sidekick.utils:is_resource_low:324 - Number of GPUs: 1
2023-12-01 01:19:19.661 | DEBUG    | sidekick.utils:is_resource_low:328 - Information on device: 0
2023-12-01 01:19:19.663 | INFO     | sidekick.utils:is_resource_low:331 - Total Memory: 23GB
2023-12-01 01:19:19.664 | INFO     | sidekick.utils:is_resource_low:332 - Free GPU memory: 9GB
2023-12-01 01:19:19.725 | INFO     | sidekick.utils:_check_file_info:438 - Using information info from path /home/pramit/testing/q/apps/system/sql-sidekick/var/lib/tmp/sleep_health_eda_demo_table_info.jsonl
2023-12-01 01:19:19.750 | DEBUG    | sidekick.utils:semantic_search:125 - Input questions: # query

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-12-01 01:19:19.795 | DEBUG    | sidekick.utils:semantic_search:130 - Probable context: ["if patterns like 'current time' or 'now' occurs in question", "if patterns like 'total number', or 'list' occurs in question", 'detailed summary', 'summary']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-12-01 01:19:19.836 | DEBUG    | sidekick.utils:semantic_search:138 - Similarity score for: if patterns like 'current time' or 'now' occurs in question: 0.8459207869447033
2023-12-01 01:19:19.839 | DEBUG    | sidekick.utils:semantic_search:138 - Similarity score for: if patterns like 'total number', or 'list' occurs in question: 0.8319947353454415
2023-12-01 01:19:19.841 | DEBUG    | sidekick.utils:semantic_search:138 - Similarity score for: detailed summary: 0.8346069603076574
2023-12-01 01:19:19.844 | DEBUG    | sidekick.utils:semantic_search:138 - Similarity score for: summary: 0.8394152180082535
2023-12-01 01:19:19.845 | DEBUG    | sidekick.utils:semantic_search:154 - Sorted context: []
2023-12-01 01:19:19.847 | DEBUG    | sidekick.query:generate_sql:402 - Filter Context: []
2023-12-01 01:19:19.848 | INFO     | sidekick.query:generate_sql:410 - Filtering Question/Query pairs ...
2023-12-01 01:19:19.850 | INFO     | sidekick.query:generate_sql:412 - Number of context queries fou

Exiting...


In [65]:
print(f"Question = {res[0][0]}")
print("----")
print(f"Generated SQL = {res[0][1]}")

question = **Generated response for question,**
What is the average sleep duration for each gender?

----
Generated SQL = ``` sql
SELECT AVG("Sleep_Duration"),
       "Gender"
FROM "sleep_health_eda_demo"
GROUP BY "Gender"
LIMIT 100
```




In [66]:
# Alternate options
res = query("What is the average sleep duration for each gender?", table_name="sleep_health_eda_demo", 
            table_info_path=table_info_path, sample_qna_path=None, regenerate_with_options=True)

2023-12-01 01:19:21.788 | INFO     | sidekick.prompter:ask:428 - Table in use: ['sleep_health_eda_demo']
2023-12-01 01:19:21.790 | INFO     | sidekick.prompter:ask:460 - Setting context...
2023-12-01 01:19:21.791 | INFO     | sidekick.prompter:ask:461 - Question: What is the average sleep duration for each gender?
2023-12-01 01:19:21.792 | INFO     | sidekick.utils:is_resource_low:324 - Number of GPUs: 1
2023-12-01 01:19:21.793 | DEBUG    | sidekick.utils:is_resource_low:328 - Information on device: 0
2023-12-01 01:19:21.794 | INFO     | sidekick.utils:is_resource_low:331 - Total Memory: 23GB
2023-12-01 01:19:21.795 | INFO     | sidekick.utils:is_resource_low:332 - Free GPU memory: 9GB
2023-12-01 01:19:21.817 | INFO     | sidekick.utils:_check_file_info:438 - Using information info from path /home/pramit/testing/q/apps/system/sql-sidekick/var/lib/tmp/sleep_health_eda_demo_table_info.jsonl
2023-12-01 01:19:21.831 | DEBUG    | sidekick.utils:semantic_search:125 - Input questions: # query

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-12-01 01:19:21.862 | DEBUG    | sidekick.utils:semantic_search:130 - Probable context: ["if patterns like 'current time' or 'now' occurs in question", "if patterns like 'total number', or 'list' occurs in question", 'detailed summary', 'summary']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-12-01 01:19:21.893 | DEBUG    | sidekick.utils:semantic_search:138 - Similarity score for: if patterns like 'current time' or 'now' occurs in question: 0.8459207869447033
2023-12-01 01:19:21.896 | DEBUG    | sidekick.utils:semantic_search:138 - Similarity score for: if patterns like 'total number', or 'list' occurs in question: 0.8319947353454415
2023-12-01 01:19:21.897 | DEBUG    | sidekick.utils:semantic_search:138 - Similarity score for: detailed summary: 0.8346069603076574
2023-12-01 01:19:21.899 | DEBUG    | sidekick.utils:semantic_search:138 - Similarity score for: summary: 0.8394152180082535
2023-12-01 01:19:21.900 | DEBUG    | sidekick.utils:semantic_search:154 - Sorted context: []
2023-12-01 01:19:21.901 | DEBUG    | sidekick.query:generate_sql:402 - Filter Context: []
2023-12-01 01:19:21.902 | INFO     | sidekick.query:generate_sql:410 - Filtering Question/Query pairs ...
2023-12-01 01:19:21.903 | INFO     | sidekick.query:generate_sql:412 - Number of context queries fou

Exiting...


In [67]:
print(f"Question = {res[0][0]}")
print("----Options----")
for _r in res[1]:
    print(_r)

question = **Generated response for question,**
What is the average sleep duration for each gender?

----Options----
Option 1: (_probability_: 0.24343933165073395)
``` sql
SELECT AVG(Gender),
       AVG(Sleep_Duration)
FROM sleep_health_eda_demo
LIMIT 100;
```



Option 2: (_probability_: 0.21723678708076477)
``` sql
SELECT Gender,
       AVG(Sleep_Duration)
FROM sleep_health_eda_demo
GROUP BY Gender
LIMIT 100;
```



Option 3: (_probability_: 0.21618281304836273)
``` sql
SELECT AVG(Sleep_Duration),
       Gender
FROM sleep_health_eda_demo
GROUP BY Gender
LIMIT 100;
```



Option 4: (_probability_: 0.17697547376155853)
``` sql
SELECT AVG(Sleep_Duration),
       Gender (TEXT)
FROM sleep_health_eda_demo
GROUP BY Gender (TEXT)
LIMIT 100;
```



Option 5: (_probability_: 0.14616547524929047)
``` sql
SELECT Gender (TEXT),
       AVG(Sleep_Duration) (NUMERIC)
FROM 'sleep_health_eda_demo'
GROUP BY Gender (TEXT)
LIMIT 100;
```





In [ ]:
# Some times, we are not sure on what Qs to ask.
import os
from sidekick.prompter import recommend_suggestions

os.environ["RECOMMENDATION_MODEL_REMOTE_URL"] = "https://playground.h2ogpte.h2o.ai"
os.environ["RECOMMENDATION_MODEL_API_KEY"] = ""

g_eqs = recommend_suggestions(table_info_path, table_name="sleep_health_eda_demo", n_qs=3)

In [ ]:
# Example 2:
# Sample QnA pair is supplied

# Given .csv file, auto-generate schema
# Replace with relevant dataset path
data_path = "<example.csv>"
# Replace table_name as needed
table_name = "test_demo"

r, table_info_path_test_demo = generate_schema(data_path, f"{cache_path}/{table_name}_table_info.jsonl")

# Set add_sample=False if no need to add rows to the table (default: = True)
llm_response, err = db_setup(
                db_name=DB_NAME,
                hostname=HOST_NAME,
                user_name=USER_NAME,
                password=PASSWORD,
                port=PORT,
                table_info_path=table_info_path_test_demo,
                table_samples_path=data_path,
                table_name=table_name,
                local_base_path=base_path
            )

In [71]:
def query(question: str, table_name, table_info_path: str, sample_qna_path: str, regenerate=False, regenerate_with_options=False):
    """Asks question and returns SQL."""
    base_path = "./"
    res = ask(
        question=question,
        table_info_path=table_info_path,
        sample_queries_path=sample_qna_path,
        table_name=table_name,
        is_command=False,
        model_name="h2ogpt-sql-sqlcoder2",
        is_regenerate=regenerate,
        is_regen_with_options=regenerate_with_options,
        execute_query=False,
        local_base_path=base_path
    )
    return res

In [ ]:
samples_path = "samples_qna_pairs.csv"
table_name = "test_demo"
# Default temperature for generation = 0.5
res = query("...", table_name=table_name, table_info_path=table_info_path_test_demo,
            sample_qna_path=samples_path)

In [ ]:
print(f"Question = {res[0][0]}")
print("----")
print(f"Generated SQL = {res[0][1]}")

In [ ]:
# With regeneration
res = query("...", table_name=table_name, table_info_path=table_info_path_test_demo,
            sample_qna_path=samples_path, regenerate=True)

In [ ]:
res = query("...", table_name=table_name, table_info_path=table_info_path_test_demo,
            sample_qna_path=samples_path, regenerate_with_options=True)

In [ ]:
print(f"Question = {res[0][0]}")
print("----")
for _r in res[1]:
    print(_r)